In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import osm_loader

bbox=[41.3752, 2.0354, 41.3889, 2.0586]

In [3]:
network = osm_loader.load_network('sant_feliu', bbox)

In [4]:
pois = osm_loader.load_pois('sant_feliu', bbox)

In [5]:
max_dist = 1500
max_pois=5

category_gb = pois.groupby(by=['key'])

network.init_pois(num_categories=len(category_gb), max_dist=max_dist, max_pois=max_pois)

In [6]:
# AUTOMATIZE THIS?
import numpy as np
from pandas import DataFrame

nearest_df = DataFrame(columns=[np.repeat([category for category, items in category_gb], max_pois), range(1, max_pois + 1) * len(category_gb)])

for category, items in category_gb:
    network.set_pois(category, items.lon, items.lat)
    [nearest_df.__setitem__((category, k), items) for k, items in network.nearest_pois(max_dist, category, num_pois=max_pois).iteritems()]

In [10]:
from geo_graph import GeoGraph

g = GeoGraph(network)

In [11]:
import networkx as nx
from pandas import Series

bc = nx.betweenness_centrality(g._G)
cc = nx.closeness_centrality(g._G)
dc = nx.degree_centrality(g._G)

centrality_df = DataFrame({'betweenness': bc, 'closeness': cc, 'degree': dc})
centrality_df.index = centrality_df.index.map(lambda geo_node: geo_node.id)

In [12]:
centrality_df

            betweenness  closeness    degree
1376136223     0.034119   0.043849  0.007092
1375635677     0.008274   0.014606  0.007092
1375635585     0.005070   0.022318  0.007092
1375635669     0.003872   0.037995  0.007092
1375635675     0.005455   0.044920  0.007092
1376116905     0.001569   0.042606  0.007092
1375635702     0.011092   0.022545  0.007092
1375635764     0.001048   0.003546  0.007092
1375635729     0.007271   0.038948  0.007092
1375635733     0.001066   0.017194  0.007092
1375635755     0.004370   0.047861  0.007092
1375635759     0.038912   0.022438  0.007092
1375635722     0.001485   0.003783  0.007092
1376136037     0.001218   0.003546  0.009456
1375635765     0.004632   0.039639  0.007092
1375635766     0.005025   0.021689  0.007092
1375635775     0.011843   0.015403  0.007092
1375635778     0.000695   0.017710  0.004728
1375635780     0.004997   0.021352  0.007092
1375635649     0.007014   0.013912  0.007092
1375635796     0.008898   0.018519  0.007092
1375657528

In [13]:
import pandas as pd

nearest_centrality_df = pd.concat([
    DataFrame({'amenity': nearest_df['amenity'][1],
     'shop': nearest_df['shop'][1]}),
    centrality_df], axis=1)
nearest_centrality_df.corr()

              amenity      shop  betweenness  closeness    degree
amenity      1.000000  0.493695    -0.135083   0.211928 -0.113215
shop         0.493695  1.000000    -0.187653   0.067806 -0.004928
betweenness -0.135083 -0.187653     1.000000   0.318083  0.320191
closeness    0.211928  0.067806     0.318083   1.000000  0.230198
degree      -0.113215 -0.004928     0.320191   0.230198  1.000000

In [14]:
%matplotlib qt

g.plot_nearest(nearest_df, 'amenity', 1)

In [10]:
%matplotlib qt

g.plot_nearest(nearest_df, 'shop', 1)

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
g = sns.PairGrid(nearest_df, diag_sharey=False, size=5)
g.map_lower(sns.kdeplot, cmap="Blues_d")
g.map_upper(plt.scatter)
# g.map_diag(sns.kdeplot, lw=3)
g.map_diag(plt.hist)